In [3]:
import pandas as pd
import numpy as np
import math

In [12]:
#define class and node
class node(): # object is required
    def __init__(self, val:int): # self is required
        self.val = val
        self.child = None  # child should be a dtree series
        self.link = None  # the link between parents and current node, represent label values here
    def __str__(self):
        return str(self.val)
# decision tree
class DTree():
    def __init__(self, val): # value = label
        self.root = node(val)
    def add(self, child:"node", link:"label"): # child is a node
        item = node(child)
        if not self.root:
            self.root = item # if there is no nodes in tree at all
            self.link = None
        else:
            self.child.append(item)
            self.lable = label

    def addchild(self, children:"list_of_node", labels:"list_of_label"):        
        for (child_item, label) in zip(children, labels):
            self.add(child_item, label)
    def setnode(self, val):
        self.val = val
    
    def printTree():
        pass
    def getchild(self):
        return self.child

In [83]:
def calIG(data, fea:"label name"):
    #this function is designed to canculate the information gain of a specific feature to the dataset, dataset: sample,feature， fea: index of the feature
    #calculate the empirical entropy of dataset d
    sample = data.shape[0]
    label = data.columns.values[-1]
    labels_len = len(np.unique(data[data.columns[-1]]))
    groups = data.groupby(data.columns.values[-1])[data.columns.values[-1]]
    hd = -sum(groups.count()/sample*np.log2(groups.count()/sample))
    groups1 = data.groupby([fea])[fea].count()
    #calculate the coditional entropy hda
    groups_fea = data.groupby([fea,label])[label].count()
    #iterate feature values
    hda=0
    for i in range(len(groups1)):
        hdi=0
        had=0 #for ig ratio
        #calculate hdi
        for j in range(labels_len):
            try:
                nums = groups_fea[i*labels_len+j]/groups1[i]
                hdi -= nums*np.log2(nums)
            except IndexError:
                hdi -= 0
        hda += groups1[i]/sample*hdi
        had -= (groups1[i] / sample*np.log2(groups1[i] / sample))
#     hda = -sum([sum(group.groupby(df.iloc[fea]).count()/group.count()*np.log2(group.groupby(df.iloc[fea]).count()/group.count())) for group in groups])
#     print('hda:',hda)
    #calculate information gain
    ig = hd - hda
    igr = ig / had
    res = {'ig':ig, 'igr':igr}
    return res

In [9]:
def calIGR(data, fea):
    #calculate the information gain ratio
    ig = calIG(data, fea)
    groups = data.groupby(df.iloc[fea])
    deno = -sum(groups.count()*np.log2(groups.count()))
    igr = ig/deno                                                     
    return igr

In [115]:
def id3(data, features, epsilon):
    # this is the id3 algorithm for decision tree
    # case 1: if all samples belong to one class, that is ,there is only one label, return a single-node tree which root is the only label ck
    labels = np.unique(data[data.columns[-1]])
    if len(labels)==1:
        return DTree(lables[0])
    # case 2: if there is no feature, return a single-node tree which root is the max ck
    if len(features)==0:
        return DTree(max(data.iloc[-1]))
    # case 3: normal case, calculate info gain of all features and choose the largest
    igs = [calIG(data,fea)['ig'] for fea in features]
    agval = max(igs)
    if agval < epsilon:
        return DTree(max(data.iloc[-1]))
    else:
        #split data according to values of features
        agind = np.where(igs==agval)
        dtree = DTree(features[agind[0][0]])
        feaval = np.unique(data.iloc[agind[0][0]])
        data_partition = data.groupby(features[agind[0][0]])
        print(data_partition.first())
        for (value, dp) in zip(feaval, data_partition):
            nodeType = max(data_partition.iloc[-1])
            dtree.setnode(nodeType)  # construct tree node
            newFeaSet = np.delete(features, agind)  # construct new feature set: fea - agind
            dtree.add(id3(dp, newFeaSet, epsilon), value)
        return dtree

In [112]:
def c45(data, features, epsilon):
    # this is the id3 algorithm for decision tree
    # case 1: if all samples belong to one class, that is ,there is only one label, return a single-node tree which root is the only label ck
    labels = np.unique(data.iloc[-1])
    if len(labels)==1:
        return DTree(lables[0])
    # case 2: if there is no feature, return a single-node tree which root is the max ck
    if len(features)==0:
        return DTree(max(data.iloc[-1]))
    # case 3: normal case, calculate info gain of all features and choose the largest
    igs = [calIG(data,fea)['igr'] for fea in range(features)]# ***************this only difference between id3 and c4.5*****************
    agval = max(igs)
    if agval < epsilon:
        return DTree(max(data.iloc[-1]))
    else:
        #split data according to values of features
        agind = np.where(igs==agval)
        dtree = DTree(features[agind])
        feaval = np.unique(data.iloc[agind])
        data_partition = data.groupby(data.iloc[agind])
        for (value, dp) in zip(feaval, data_partition):
            nodeType = max(data_partition.iloc[-1])
            dtree.setnode(nodeType)  # construct tree node
            newFeaSet = np.delete(features, agind)  # construct new feature set: fea - agind
            dtree.add(c45(dp, newFeaSet, epsilon), value)
        return dtree

In [116]:
# load data
train = pd.read_csv('./test1.csv')
train = train.drop(['ID'],axis=1)
# print(train.groupby(['有房子','类别'])['类别'].count()[1])
print(train.columns.values)
dt = id3(train,train.columns.values[0:-1],0.1)

['年龄' '有工作' '有房子' '信贷情况' '类别']
     年龄 有工作 信贷情况 类别
有房子                
否    青年   否   一般  否
是    青年   是   一般  是


AttributeError: Cannot access callable attribute 'iloc' of 'DataFrameGroupBy' objects, try using the 'apply' method